# define the COBRA model

In [1]:
# define the environment path 
import os
local_cobrakbase_path = os.path.join('C:', 'Users', 'Andrew Freiburger','Documents','Argonne','cobrakbase')
os.environ["HOME"] = local_cobrakbase_path

import cobrakbase
token = 'MIXOUVGIFOUPQD2KSH6DQ5ARKDLBARCL'
kbase = cobrakbase.KBaseAPI(token)

model = kbase.get_from_ws("CMM_iAH991V2_iML1515.kb",40576)
media = kbase.get_from_ws("Btheta_Ecoli_minimal_media",40576)

cobrakbase 0.2.8


# define Python execution function

In [7]:
def execute_julia(cobra_model):
    # setup Julia
    from julia.api import Julia
    jl = Julia(compiled_modules=False)
    from julia import Main
    # jl.using("TextAnalysis")
    # jl.eval('include("cobra_execution.jl")')
    Main.include("cobra_execution.jl")
    
    # load the model
    # from cobra.io import save_matlab_model
    # model_path = 'test.mat'
    # save_matlab_model(model, model_path)
    # Main.model_path = model_path
    # from modelseedpy import BaseFBAPkg
    # BaseFBAPkg(model, 'test').print_lp('test')
    from cobra.io import save_json_model
    save_json_model(model, 'test.json')
    Main.execute_cobra('test.lp')
    # solution = jl.eval('execute_cobra(model_path)')
    
    # remove the MATLAB file
    from os import remove
    remove(model_path)
    
    return solution

## Python time

In [3]:
%time python_sol = model.optimize()

CPU times: total: 500 ms
Wall time: 497 ms


## Julia time

In [4]:
%time julia_sol = execute_julia(model)

# execute_cobra('test.mat')

ImportError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: LoadError: InitError: could not load library "libmbedtls.dll"
The file cannot be accessed by the system. 
Stacktrace:
  [1] dlopen(s::String, flags::UInt32; throw_error::Bool)
    @ Base.Libc.Libdl .\libdl.jl:117
  [2] dlopen (repeats 2 times)
    @ .\libdl.jl:117 [inlined]
  [3] __init__()
    @ MbedTLS_jll C:\Users\Andrew Freiburger\AppData\Local\Programs\Julia-1.7.3\share\julia\stdlib\v1.7\MbedTLS_jll\src\MbedTLS_jll.jl:42
  [4] include
    @ .\Base.jl:418 [inlined]
  [5] _require(pkg::Base.PkgId)
    @ Base .\loading.jl:1149
  [6] require(uuidkey::Base.PkgId)
    @ Base .\loading.jl:1013
  [7] require(into::Module, mod::Symbol)
    @ Base .\loading.jl:997
  [8] include
    @ .\Base.jl:418 [inlined]
  [9] _require(pkg::Base.PkgId)
    @ Base .\loading.jl:1149
 [10] require(uuidkey::Base.PkgId)
    @ Base .\loading.jl:1013
 [11] require(into::Module, mod::Symbol)
    @ Base .\loading.jl:997
 [12] include(mod:

In [8]:
%time julia_sol = execute_julia(model)

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: UndefVarError: read_from_file not defined
Stacktrace:
 [1] execute_cobra(model_path::String)
   @ Main C:\Users\Andrew Freiburger\Documents\Argonne\ProjectNotebooks\CommunityModeling\julia\cobra_execution.jl:23
 [2] invokelatest(::Any, ::Any, ::Vararg{Any}; kwargs::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
   @ Base .\essentials.jl:716
 [3] invokelatest(::Any, ::Any, ::Vararg{Any})
   @ Base .\essentials.jl:714
 [4] _pyjlwrap_call(f::Function, args_::Ptr{PyCall.PyObject_struct}, kw_::Ptr{PyCall.PyObject_struct})
   @ PyCall C:\Users\Andrew Freiburger\.julia\packages\PyCall\7a7w0\src\callback.jl:28
 [5] pyjlwrap_call(self_::Ptr{PyCall.PyObject_struct}, args_::Ptr{PyCall.PyObject_struct}, kw_::Ptr{PyCall.PyObject_struct})
   @ PyCall C:\Users\Andrew Freiburger\.julia\packages\PyCall\7a7w0\src\callback.jl:44>